# R&D or PRODUCTION phase: This will generate a PIPELINE with 1-M Databricks steps
- All Databricks steps, or mixed with Python steps
- Purpose: Creates 1 of the 2 PIPELINES
    - `2a) training pipeline:` TRAINS a model with Azure AutoML and with AZURE compute cluster and calculates test_set scoring, automatically compares if newly trained model is better. 

# Prerequisite - Databricks:
- You need to have a `ESML Databricks template snaphshot folder` (M01,M11, ...) in your Databricks workspace
- Run the notebooks in that folder first, interactively, so you know they work - then you come back to THIS notebooks, to generate an Azure ML pipeline, pointing at those Databricks notebooks

# TODO for you: CONFIGURATION
- 1) Change `p.active_model=11` to correct model number `1` if your model has that number.
    - See  [lake_settings.json](./settings/project_specific/model/lake_settings.json) to find YOUR model number.
- 2) After you run the cell [2) AUTO-GENERATE code: a snapshot folder](#2_generate_snapshot_folder), you need to add YOUR feature engineering logic
    -  This code you probably already have, from the R&D phase, in this CUSTOMIZE cell in the notebook: [1_R&D_phase_M10_M11.ipynb](./1_quickstart/1_R&D_phase_M10_M11.ipynb)
        - 2a) You need to add this code to the `your_custom_code.py` after you have genereated the snapshot folder, for it to be reachable and uploaded at pipeline creation.
            - Tip: You can CREATE A CLASS, and add static methods, e.g. `ds01_process_in2silver(dataframe1)`  in the `your_custom_code.py` 
        - 2b) You need to create the Databricks mapping here `ESMLPipelineStepMap.py`, define which steps are going to be handled by a Databricks notebooks and Databricks Sparl cluster.
- 3) Now you have your code in the `your_custom_code.py`, then you need to reference that code from the auto-generated pipeline-steps files such as `in2silver_ds01_diabetes.py`
    - Note: This snapshot folder will not exist, until you have run the first 2 cells in this notebook, or after this cell has run [2) AUTO-GENERATE code: a snapshot folder](#2_generate_snapshot_folder)

## 1) Initiate ESMLPipelineFactory (Always run thic CELL below)
- To attach ESML controlplane to your project
- To point at `template-data` for the pipelinbe to know the schema of data
- To init the ESMLPipelinefactory

In [1]:
import sys
sys.path.insert(0, "../azure-enterprise-scale-ml/esml/common/")
from esml import ESMLProject
from baselayer_azure_ml_pipeline import ESMLPipelineFactory, esml_pipeline_types

p = ESMLProject() # Will search in ROOT for your copied SETTINGS folder '../settings/model/active/active_in_folder.json',
p.inference_mode = False
p.active_model = 11 # 10=titanic , 11=Diabetes
p.ws = p.get_workspace_from_config()
p_factory = ESMLPipelineFactory(p)

training_datefolder = '1000-01-01 10:35:01.243860' # Will override active_in_folder.json
p_factory.batch_pipeline_parameters[0].default_value = 0 # Will override active_in_folder.json.model.version = 0 meaning that ESML will find LATEST PROMOTED, and not use a specific Model.version. It will read data from .../inference/0/... folder
p_factory.batch_pipeline_parameters[1].default_value = training_datefolder # overrides ESMLProject.date_scoring_folder.
p_factory.describe()

all_steps_databricks = False #Notebook parameter: Disabled CELL that includes all mapped steps as DatabricksSteps
simple_mode_but_separate_compute = False


Using lake_settings.json with ESML version 1.4 - Models array support including LABEL

 ---- Q: WHICH files are generated as templates, for you to EDIT? ---- 
A: These files & locations:
File to EDIT (step: IN_2_SILVER_1): ../../../01_pipelines/batch/M11/in2silver_ds01_diabetes.py
File to EDIT (step: IN_2_SILVER_2): ../../../01_pipelines/batch/M11/in2silver_ds02_other.py
File to EDIT (step: SILVER_MERGED_2_GOLD): ../../../01_pipelines/batch/M11/silver_merged_2_gold.py
File to EDIT (step: SCORING_GOLD): ../../../01_pipelines/batch/M11/scoring_gold.py
File to EDIT (step: TRAIN_SPLIT_AND_REGISTER): ../../../01_pipelines/batch/M11/train_split_and_register.py
File to EDIT (step: TRAIN_MANUAL): ../../../01_pipelines/batch/M11/train_manual.py
File to EDIT (step: TRAIN_AUTOML): ../../../01_pipelines/batch/M11/train_post_automl_step.py
File to EDIT a lot (reference in step-scripts Custom code): ../../../01_pipelines/batch/M11/your_code/your_custom_code.py

 ---- WHAT model to SCORE with, & WHAT

# The below cells for an IN_2_GOLD_TRAIN_AUTOML pipeline will:
- 1) Generate code files
- 2) Build pipeline, ESML autoguild this, and will upload the snapshot folder together with the Azure ML pipeline.
- 3) Run the pipeline. Smoke testing, see that it works
- 4) IF it works, Publish the pipeline, or else, edit the code files or configuration, retry step 2 and 3.
- 5) Print the pipeline_id, that is essential to use from Azure Data factory 

# 2) `AUTO-GENERATE code: a snapshot folder`
<a id='2_generate_snapshot_folder'></a>

In [2]:
## Generate CODE - then edit it to get correct environments
p_factory.create_dataset_scripts_from_template(overwrite_if_exists=False) # Do this once, then edit them manually. overwrite_if_exists=False is DEFAULT

Did NOT overwrite script-files with template-files such as 'scoring_gold.py', since overwrite_if_exists=False


## Alternative A) - Filter out, use some steps, a whitelist
- Using a whitelist filter. 1-M of your mapped steps

In [3]:
import sys
sys.path.insert(0, "../azure-enterprise-scale-ml/")
from esmlrt.interfaces.iESMLPipelineStepMap import IESMLPipelineStepMap
from esmlrt.interfaces.iESMLPipelineStepMap import esml_snapshot_step_names
sys.path.insert(0, "../01_pipelines/batch/M11/your_code/")
from ESMLPipelineStepMap import ESMLPipelineStepMap

dataset_folder_names = p.active_model['dataset_folder_names']
step1 = esml_snapshot_step_names.in2silver_template.format(dataset_folder_names[0])
step2 = esml_snapshot_step_names.in2silver_template.format(dataset_folder_names[1])
step3 = esml_snapshot_step_names.silver_merged_2_gold
step4 = esml_snapshot_step_names.train_split_and_register
step5 = esml_snapshot_step_names.train_manual

step_filter_whitelist = [step1,step2,step3]

my_map = ESMLPipelineStepMap(step_filter_whitelist) # TODO 4 YOU: You need to implement this class. See "your_code" folder 
#map = ESMLPipelineStepMap()
p_factory.use_advanced_compute_settings(my_map)

# Print the Mappning
train_map = my_map.get_train_map(p.active_model['dataset_folder_names']) # Get the map
train_map # prints it

Azure ML Workspace:
Attached Databricks db_compute_name:
Compute target n1-p002-aml-91 already exists


[{'step_name': 'in2silver_ds01_diabetes',
  'code': '/Repos/jostrm@microsoft.com/esml-mlops/notebook_databricks/esml/dev/project/11_diabetes_model_reg/M11/10_in2silver_ds01_diabetes',
  'compute_type': 'dbx',
  'date_folder_or': None,
  'dataset_folder_names': 'ds01_diabetes',
  'dataset_filename_ending': '*.csv',
  'compute_name': 'n1-p002-aml-91',
  'cluster_id': '1201-224730-rn6t5arn'},
 {'step_name': 'in2silver_ds02_other',
  'code': '/Repos/jostrm@microsoft.com/esml-mlops/notebook_databricks/esml/dev/project/11_diabetes_model_reg/M11/10_in2silver_ds02_other',
  'compute_type': 'dbx',
  'date_folder_or': None,
  'dataset_folder_names': 'ds02_other',
  'dataset_filename_ending': '*.csv',
  'compute_name': 'n1-p002-aml-91',
  'cluster_id': '1201-224730-rn6t5arn'},
 {'step_name': 'silver_merged_2_gold',
  'code': '/Repos/jostrm@microsoft.com/esml-mlops/notebook_databricks/esml/dev/project/11_diabetes_model_reg/M11/20_merge_2_gold',
  'compute_type': 'dbx',
  'date_folder_or': None,
  

## Alternative B) - Use all possible steps you defined in the ESMLPipleineStepMap
- No whitelist filter

In [4]:
import sys
sys.path.insert(0, "../azure-enterprise-scale-ml/")
from esmlrt.interfaces.iESMLPipelineStepMap import IESMLPipelineStepMap
sys.path.insert(0, "../01_pipelines/batch/M11/your_code/")
from ESMLPipelineStepMap import ESMLPipelineStepMap

if(all_steps_databricks):
    mapping = ESMLPipelineStepMap() # TODO 4 YOU: You need to implement this class. See "your_code" folder 
    p_factory.use_advanced_compute_settings(mapping)

    # Print the Mappning
    train_map = mapping.get_train_map(p.active_model['dataset_folder_names']) # Get the map
    train_map # prints it
else:
    print("This notebook CELL is disabled. Change 'all_steps_databricks=True' to enable it.")

This notebook CELL is disabled. Change 'all_steps_databricks=True' to enable it.


# TRAINING (3a,4a,5a)

# 3a) `BUILDS the TRANING pipeline`
- esml_pipeline_types.IN_2_GOLD_TRAIN_AUTOML
- Take note on the `esml_pipeline_types` below, of type: esml_pipeline_types.`IN_2_GOLD_TRAIN_AUTOML`

In [5]:
## BUILD (takes ~6-12minutes)
if(simple_mode_but_separate_compute):
    p_factory.use_advanced_compute_settings(None)
    batch_pipeline = p_factory.create_batch_pipeline(esml_pipeline_types.IN_2_GOLD_TRAIN_MANUAL, same_compute_for_all=False, aml_compute=None, allow_reuse=True)
else:
    batch_pipeline = p_factory.create_batch_pipeline(esml_pipeline_types.IN_2_GOLD_TRAIN_MANUAL)
    #batch_pipeline = p_factory.create_batch_pipeline(esml_pipeline_types.IN_2_GOLD_TRAIN_MANUAL, same_compute_for_all=True, aml_compute=None, allow_reuse=True)

Using GEN2 as Datastore
Environment ESML-AzureML-144-AutoML_126 exists
Dataset: ds01_diabetes has advanced mapping - an Azure Databricks mapping
Dataset: ds02_other has advanced mapping - an Azure Databricks mapping
ESML advanced mode: with advanced compute mappings
 - Step: silver_merged_2_gold has advanced mapping - an Azure Databricks mapping
Found attached Databricks compute cluster
previous_step_is_databricks = 1
create_gold_train_step: inference_mode=False
par_date_utc: 1000-01-01 10:35:01.243860
Created Databricks step in pipeline
Initiated DEFAULT compute - for step train_split_and_register 
ESML will auto-create a compute...
Note: OVERRIDING enterprise performance settings with project specifics. (to change, set flag in 'dev_test_prod_settings.json' -> override_enterprise_settings_with_model_specific=False)
Using a model specific cluster, per configuration in project specific settings, (the integer of 'model_number' is the base for the name)
Note: OVERRIDING enterprise perform

## 4a) `EXECUTES the pipeline`

### NB! Run in v1 legacy mode
- You need to have your Azure Machine Learning workspace set to `v1_legacy_mode=True`
- HOW do I know if I run v1 or v2? 
  - If you see this error message in `executionlogs.txt in Azure machine learning studio Output+logs tab on pipeline rune`, containing the word in path `backendV2` when executing pipeline (cell below this), it is not in v1 legacy mode:
     - <i>Failed to start the job for runid: 33ff1e3a-1ca7-4de0-bcee-b851cd2bb89d because of exception_type: ServiceInvocationException, error: Failure in StartSnapshotRun while calling service Execution; HttpMethod: POST; Response StatusCode: BadRequest; Exception type: Microsoft.RelInfra.Extensions.HttpRequestDetailException|-Microsoft.RelInfra.Common.Exceptions.ErrorResponseException, stack trace:    at Microsoft.Aether.EsCloud.Common.Client.ExecutionServiceClient.StartSnapshotRunAsync(String jobId, RunDefinition runDefinition, String runId, WorkspaceIdentity workspaceIdentity, String experimentName, CreatedBy createdBy) in D:\a\_work\1\s\src\aether\platform\\`backendV2`\\Clouds\ESCloud\ESCloudCommon\Client\ExecutionServiceClient.cs:line 162
   at Microsoft.Aether.EsCloud.Common.JobProcessor.StartRunAsync(EsCloudJobMetadata job) in D:\a\_work\1\s\src\aether\platform\backendV2\Clouds\ESCloud\ESCloudCommon\JobProcessor.cs:line 605
   </i>
- WHY? 
    - Azure ML SDK v2 does not yet (writing this 2022-10)support Spark jobs in pipeline, nor private endpoint.
- TODO: To set the workspace in LEGACY v1 mode run this code 1 time, in a cell: `p.ws.update(v1_legacy_mode=True)`

In [6]:
#p.ws.update(v1_legacy_mode=True) # If you happen to have a workspace in v2 mode, and want to change back to v1 legacy mode

label = 'Y'
train_df = aml.to_pandas_dataframe()
#y1 = train_df[label]
X=train_df.drop(label, axis=1)
y = train_df.pop(label).to_frame()

#print(y1.head()) # no column
#print(type(y1)) # series

print(X.head())
print("")
print(y.head())

In [7]:
p.GoldPathDatabricks

'projects/project002/11_diabetes_model_reg/train/gold/dev/gold_dbx.parquet/*.parquet'

In [8]:
train_path,validate_path,test_path = p.path_gold_train_splitted_template() # projects/project002/11_diabetes_model_reg/train/gold/dev/Train/{id_folder}}/
train_path_out = train_path.format(id_folder="gold_train_dbx.parquet") + '*.parquet' # gold_train_dbx.parquet
print(train_path_out)

projects/project002/11_diabetes_model_reg/train/gold/dev/Train/gold_train_dbx.parquet/*.parquet


In [9]:
for s in p_factory.pipeline_steps_array:
    print(type(s))

<class 'azureml.pipeline.steps.databricks_step.DatabricksStep'>
<class 'azureml.pipeline.steps.databricks_step.DatabricksStep'>
<class 'azureml.pipeline.steps.databricks_step.DatabricksStep'>
<class 'azureml.pipeline.steps.python_script_step.PythonScriptStep'>
<class 'azureml.pipeline.steps.python_script_step.PythonScriptStep'>


In [10]:
## RUN and it will train in BIG Data, since using 100% Azure compute for all steps, including SPLITTING data
pipeline_run = p_factory.execute_pipeline(batch_pipeline) # If this give ERROR message, looking at executionlogs.txt in Azure machine learning studio Output+logs tab on pipeline rune
pipeline_run.wait_for_completion(show_output=False)

Execute_pipeline (scoring): Inference_mode: 0
-Scoring data, default value 1000-01-01 10:35:01.243860
Adding pipeline parameters
Created step in2silver_ds01_diabetes [abd9c113][f7347261-2bfb-4895-81f4-670128a9b1ce], (This step will run and generate new outputs)Created step in2silver_ds02_other [2c845737][dfa6d432-2f6f-481e-b63c-7f4de9c8ee68], (This step will run and generate new outputs)

Created step silver_merged_2_gold [20a1fa9e][a8dbabba-e4e5-472e-b02a-e1d1a9591280], (This step will run and generate new outputs)
Created step SPLIT AND REGISTER datasets [cb93096b][5344bbe8-b056-4ea7-8d44-4ac5faa5e82c], (This step will run and generate new outputs)
Created step TRAIN in  [dev], COMPARE & REGISTER model in [dev] & PROMOTE to [test] [8ef8174e][ce64b3e6-feaa-4ebc-be30-b14cfa3548dc], (This step will run and generate new outputs)
Created data reference M11_ds01_diabetes_train_IN for StepId [86d25f21][a16432db-47a4-4091-8f49-ebf6f30d6944], (Consumers of this data will generate new runs.)Cr

'Finished'

# 5a) PUBLISH the TRAINING pipeline & PRINT its ID

In [ ]:
# PUBLISH
published_pipeline, endpoint = p_factory.publish_pipeline(batch_pipeline,"_1") # "_1" is optional    to create a NEW pipeline with 0 history, not ADD version to existing pipe & endpoint

# PRINT: Get info to use in Azure data factory
- `published_pipeline.id` (if private Azure ML workspace)

In [ ]:
print("2) Fetch scored data: Below needed for Azure Data factory PIPELINE activity (Pipeline OR Endpoint. Choose the latter") 
print ("- Endpoint ID")
print("Endpoint ID:  {}".format(endpoint.id))
print("Endpoint Name:  {}".format(endpoint.name))
print("Experiment name:  {}".format(p_factory.experiment_name))

print("In AZURE DATA FACTORY - This is the ID you need, if using PRIVATE LINK, private Azure ML workspace.")
print("-You need PIPELINE id, not pipeline ENDPOINT ID ( since cannot be chosen in Azure data factory if private Azure ML)")
published_pipeline.id

# DONE! Next Step - Deploy model, serve your model for INFERENCING purpose:
- For INFERENCE you may need either to DEPLOY the model 
    - a) ONLINE on AKS endpoint
        - Notebook: 
    - b) BATCH SCORING on an Azure machine learning pipeline
        - Notebook: [your_root]\notebook_templates_quickstart\\`3a_PRODUCTION_phase_BATCH_INFERENCE_Pipeline.ipynb`
    - c) STREAMING using Eventhubs and Azure Databricks structured streaming
        - Notebook: TBA

- Q: `Next step in PRODUCTION phaase after the 2a and 3a or 3b notebooks are done?`
 
- 1) `DataOps+MLOps:` Go to your ESMLProjects `Azure data factory`, and use the `ESML DataOps templates` (Azure data factory templates) for `IN_2_GOLD_TRAIN` and `IN_2_GOLD_SCORING`
    - azure-enterprise-scale-ml\copy_my_subfolders_to_my_grandparent\adf\v1_3\PROJECT000\LakeOnly\\`STEP03_IN_2_GOLD_TRAIN_v1_3.zip`
- 2) `MLOps CI/CD` Go to the next notebook `mlops` folder, to setup `CI/CD` in Azure Devops
    - Import this in Azure devops
        azure-enterprise-scale-ml\copy_my_subfolders_to_my_grandparent\mlops\01_template_v14\azure-devops-build-pipeline-to-import\\`ESML-v14-project002_M11-DevTest.json`
    - Change the Azure Devops `VARIABLES` for service principle, tenant, etc.
    - Change parameters in the `inlince Azure CLI script` to correct model you want to work with, and the correct data you want to train with, or score.
        - Step `21-train_in_2_gold_train_pipeline`
        - INLINE code calls the file: `21-train_in_2_gold_train_pipeline.py`
        - INLINE parameters: `--esml_model_number 11 --esml_date_utc "1000-01-01 10:35:01.243860"`

# StepMap - how to print & look at it?

In [ ]:
train_map = map.get_train_map(p.active_model['dataset_folder_names'])
has_dbx,step_name,map_step = map.get_dbx_map_step(train_map,'ds01_diabetes')
print(has_dbx)
print(step_name)

In [ ]:
#train_map = map.get_train_map(p.active_model['dataset_folder_names'])
for d in p.Datasets:
    print(d.Name)
    has_dbx,step_name,map_step = map.get_dbx_map_step(train_map,d.Name)
    print("has_dbx:",has_dbx)
    print("step_name",step_name)
    print("")